
### Expected flow of program

#### Module 01 : Get Statistics

    1. Get the root directory to traverse.
    2. Recursively generate md5 for all files in root, sub directories. Dump the result in csv.

#### Module 02 : Move Files

    1. Analyze the result stored in csv.
    2. Prepare the source and destination directory structure in excel.
    3. Run python to read excel and move the files as mentioned in excel.




<hr style="border: solid rgb(255,255,255) 1.0px;height: 2.0px;"/>

#### Module 01 : Get Statistics

In [3]:
# https://github.com/UrviSoni/remove_duplicate_image/blob/master/duplicate_image_remove.ipynb

import hashlib
from skimage.io import imread
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import time
import numpy as np
import logging
import datetime
import hashlib
import os
import sys
import csv
import filetype
import pandas as pd

%matplotlib inline

In [11]:
# Set up program level variables

searchDir = 'S:\Pictures\duplicateFileFinder\duplicates\\' 
destinDir = 'S:\Pictures\duplicateFileFinder\duplicates\\' 
outDir = 'S:\Pictures\duplicateFileFinder\logs\\'
outFile = outDir + time.strftime("%Y%m%d_%H%M%S") + '.csv'

logDir = 'S:\Pictures\duplicateFileFinder\logs\\'
logFile = logDir + time.strftime("%Y%m%d_%H%M%S") + '.log'

logging.basicConfig(filename= logFile, level=logging.INFO)

logging.info(time.strftime("\n\n\n%Y%m%d_%H%M%S") + ' : ******* Program Start  ******* ')
logging.info(time.strftime("%Y%m%d_%H%M%S") + ' : searchDir : ' + searchDir)
logging.info(time.strftime("%Y%m%d_%H%M%S") + ' : destinDir : ' + destinDir)
logging.info(time.strftime("%Y%m%d_%H%M%S") + ' : outDir : ' + outDir)
logging.info(time.strftime("%Y%m%d_%H%M%S") + ' : outFile : ' + outFile)
logging.info(time.strftime("%Y%m%d_%H%M%S") + ' : logDir : ' + logDir)
logging.info(time.strftime("%Y%m%d_%H%M%S") + ' : logFile : ' + logFile)

In [12]:
walk_dir = searchDir

duplicates = []
hash_keys = dict()

with open(outFile, mode='w', newline='') as csvFile:
    fieldnames = ['file_path', 'filehash']
    
    # creating a csv writer object 
    csvwriter = csv.writer(csvFile) 
        
    # writing the fields 
    csvwriter.writerow(fieldnames) 
    
    index = 0
    for root, subdirs, files in os.walk(walk_dir):
        for filename in files:
            file_path = os.path.join(root, filename)
            
            if filetype.is_image(file_path) or filetype.is_video(file_path):
                # generate and dump md5 to csv
                with open(file_path, 'rb') as f:
                    filehash = hashlib.md5(f.read()).hexdigest()
                
                '''
                index += 1
                if filehash not in hash_keys: 
                    hash_keys[filehash] = index
                else:
                    #duplicates.append((index,hash_keys[filehash]))
                    duplicates.append((file_path))
                '''
            else:
                filehash = "not_an_image_or_video"
            currRow = [[file_path, filehash]]
            csvwriter.writerows(currRow) # encoding='cp1252'


In [ ]:
print(duplicates)

In [5]:
df = pd.read_csv('S:\\Pictures\\duplicateFileFinder\\logs\\20210406_210031.csv')

In [6]:
df.head(5)

,file_path,filehash
0,S:\Pictures\duplicateFileFinder\logs\20210406_...,not_an_image_or_video
1,S:\Pictures\duplicateFileFinder\logs\20210406_...,not_an_image_or_video
2,S:\Pictures\iPhone12\20210220\Picture\IMG_6372...,0b3a067c03ccb25dfa05050f7a868b4f
3,S:\Pictures\iPhone12\20210220\Picture\IMG_6372...,0b3a067c03ccb25dfa05050f7a868b4f
4,S:\Pictures\iPhone12\20210220\Picture\IMG_6376...,7a3ddced9b5cbe1bc0f2f12f08e989a7


In [8]:
duplicateRowsDF = df[df.duplicated(subset=['filehash'])]

In [9]:
# Duplicate Rows except first occurrence
print(duplicateRowsDF)

                                              file_path  \
1     S:\Pictures\duplicateFileFinder\logs\20210406_...   
3     S:\Pictures\iPhone12\20210220\Picture\IMG_6372...   
239           S:\Pictures\iPhone8\101APPLE\CPZA2523.JPG   
294           S:\Pictures\iPhone8\101APPLE\IMG_1353.AAE   
335           S:\Pictures\iPhone8\101APPLE\IMG_1415.AAE   
...                                                 ...   
6128      S:\Pictures\iPhone8Plus\101APPLE\IMG_1976.AAE   
6130  S:\Pictures\iPhone8Plus\101APPLE\IMG_1977 (760...   
6131      S:\Pictures\iPhone8Plus\101APPLE\IMG_1977.AAE   
6133  S:\Pictures\iPhone8Plus\101APPLE\IMG_1978 (761...   
6134      S:\Pictures\iPhone8Plus\101APPLE\IMG_1978.AAE   

                              filehash  
1                not_an_image_or_video  
3     0b3a067c03ccb25dfa05050f7a868b4f  
239   54a1b947e8664bbbb800040e8218ab5b  
294              not_an_image_or_video  
335              not_an_image_or_video  
...                                ...  
6128

In [10]:
duplicateRowsDF.to_csv(r'S:\Pictures\duplicateFileFinder\logs\20210406_210031 - duplicates.csv')

<hr style="border: solid rgb(255,255,255) 1.0px;height: 2.0px;"/>

#### Module 02 : Move Files

In [4]:
for dupFile in duplicates:
    
    sourceFile = dupFile
    destinFile = destinDir + "\\".join([]) + os.path.basename(dupFile)
    
    try:
        if os.path.isfile(sourceFile):
            
            destinFile_woExt = os.path.splitext(destinFile)[0]
            destinFile_Extn = os.path.splitext(destinFile)[1]
            expand = 0
            while True:
                expand += 1
                new_file_name = destinFile_woExt + " - " + str(expand) + destinFile_Extn
                if os.path.isfile(new_file_name):
                    continue
                else:
                    destinFile = new_file_name
                    break
            os.rename(sourceFile, destinFile)
            logging.info(time.strftime("%Y%m%d_%H%M%S") + ' : SUCCESS : Moved : ' + sourceFile + ' | '+ destinFile)
    except:
        logging.info(time.strftime("%Y%m%d_%H%M%S") + ' : ERROR : Moving : ' + sourceFile + ' | '+ destinFile)
 

<hr style="border: solid rgb(255,255,255) 1.0px;height: 2.0px;"/>

#### Module 03 : End

In [5]:
logging.info(time.strftime("%Y%m%d_%H%M%S") + ' ******* Program End  ******* ')

<hr style="border: solid rgb(255,255,255) 1.0px;height: 2.0px;"/>